In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from scipy.stats import t

sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'grid.linestyle':'--'})

## Automobile MPG data

In this class, we will use the "Auto MPG" dataset for demonstrative purposes.

In [5]:
data = pd.read_csv("https://raw.githubusercontent.com/changyaochen/MECE4520/master/data/auto_mpg.csv")
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car
0,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
1,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
2,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
3,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
4,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino


### Data representation

It is very common to represent the data in the matrix format, and with `pandas`, we can extract the underlying data as `numpy.ndarray`. 

In [7]:
# Retrieve data as a numpy array
Y = data[["mpg"]].values
print(Y.shape)
Y[:5]

(398, 1)


array([[18.],
       [15.],
       [18.],
       [16.],
       [17.]])

### Simple linear regression

In simple linear regression, there is only one independent variable. The goal of the (simple) linear regression, is to find the best values of coefficients, $\beta$, that minimize the error sum of squares. 

To find $\beta$, we can simply apply calculus, via the so-call Normal equation.

In [8]:
X = np.append(np.ones(shape=(data.shape[0], 1)), data[["weight"]].values, axis=1)
print(X.shape)
X[:5]

(398, 2)


array([[1.000e+00, 3.504e+03],
       [1.000e+00, 3.693e+03],
       [1.000e+00, 3.436e+03],
       [1.000e+00, 3.433e+03],
       [1.000e+00, 3.449e+03]])

In [9]:
# Calculates coefficients
betas = np.linalg.inv(X.T @ X) @ X.T @ Y
betas

array([[ 4.63173644e+01],
       [-7.67661006e-03]])

In [10]:
# Calculates the standard error
Y_hat = X @ betas
residual = Y - Y_hat
var = np.var(residual, ddof=X.shape[1])

se = np.sqrt(var * np.linalg.inv(X.T @ X))
se

/var/folders/ck/6xgx96j15vd6tcwzl68twbpc0000gn/T/ipykernel_77486/1414136738.py:6: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(var * np.linalg.inv(X.T @ X))


array([[7.95245230e-01,            nan],
       [           nan, 2.57486863e-04]])

In [11]:
# Calculates R2
r2 = np.power(Y_hat - np.mean(Y), 2).sum() / np.power(Y - np.mean(Y), 2).sum()
r2

0.6917929800341555

### Confidence interval and prediction interval

Both confidence intervals and prediction intervals are used to convey the uncertainty associated with estimates or predictions from regression models, but they serve different purposes and are interpreted differently.

#### Confidence Interval (CI):

A confidence interval provides a range of values for some parameter (e.g., the mean or a regression coefficient) based on the data from a sample. It captures the uncertainty about the parameter estimate in repeated sampling. If we were to collect **many samples** and compute a confidence interval from each of them, we expect a certain proportion (e.g., 95% for a 95% CI) of those intervals to contain the true parameter value. For a 95% confidence interval, we'd say that we are 95% confident that the true parameter value lies within this interval.

#### Prediction Interval (PI):

A prediction interval provides a range within which **a new individual observation** is expected to fall with a certain probability, given what we know from a set of data.
It captures both the uncertainty associated with the estimated regression line (or mean) and the variability of individual data points around that line.
For a 95% prediction interval, we'd say that we are 95% confident that a new individual observation will fall within this interval.

In [ ]:
# confidence interval
x_new = 4000

y_hat = (betas[0] + betas[1] * x_new)[0]
print(y_hat)

delta = np.sqrt(var) * np.sqrt(1 / X.shape[0] + (x_new - np.mean(X[:, 1]))**2 / np.sum((X[:, 1] - np.mean(X[:, 1]))**2))  
multiplier = 1.96
# multiplier = t.ppf(q=0.975, df=X.shape[0] - X.shape[1])

print(f"The lower bound of the 95% CI is: {y_hat - multiplier * delta:5.3f}")
print(f"The upper bound of the 95% CI is: {y_hat + multiplier * delta:5.3f}")

In [ ]:
delta = np.sqrt(var) * np.sqrt(1 + 1 / X.shape[0] + (x_new - np.mean(X[:, 1]))**2 / np.sum((X[:, 1] - np.mean(X[:, 1]))**2))  

print(f"The lower bound of the 95% PI is: {y_hat - multiplier * delta:5.3f}")
print(f"The upper bound of the 95% PI is: {y_hat + multiplier * delta:5.3f}")

### Multiple linear regression

For the general, namely, multiple linear regression, we consider more than one independent variable. All the concepts from the simple linear regression still apply.

In [ ]:
X = data[["weight", "acceleration"]].values
X = np.append(np.ones((X.shape[0], 1)), X, axis=1)
print(X.shape)
X[:5]

In [ ]:
# Calculates coefficients
betas = np.linalg.inv(X.T @ X) @ X.T @ Y
betas

In [ ]:
# Calculates the standard error
Y_hat = X @ betas
residual = Y - Y_hat
var = np.var(residual, ddof=X.shape[1])

se = np.sqrt(var * np.linalg.inv(X.T @ X))
se

In [ ]:
# Calculates R2
r2 = np.power(Y_hat - np.mean(Y), 2).sum() / np.power(Y - np.mean(Y), 2).sum()
r2

In [ ]:
# Linear regression with the `statsmodels` library
model_1 = smf.ols(formula='mpg ~ weight + acceleration', data=data)
result_1 = model_1.fit()
print(result_1.summary())

In [ ]:
# One-hot encode the categorical variables
model_2 = smf.ols(formula='mpg ~ weight + C(origin)', data=data)
result_2 = model_2.fit()
print(result_2.summary())

### Multicollinearity 

Multicollinearity refers to a situation in multiple (linear) regression where two or more independent variables (predictors) are highly correlated with each other. This means that they have similar patterns of variance and are providing redundant information in predicting the dependent variable.

#### Consequences of Multicollinearity:

* Coefficients Become Unstable: The estimated coefficients become sensitive to small changes in the model. A slight change in the data can lead to large changes in the coefficients.
* Uncertainty in Coefficients: The standard errors of the coefficients increase, which means the coefficients might not be statistically significant even if they truly have an effect.
* Interpretation Issues: Because the variables are correlated, it becomes difficult to disentangle the individual effect of one predictor from the effects of other correlated predictors.
* Decreased Model Generalizability: A model with multicollinearity may not generalize well to new data.

In [12]:
# Multicollinearity
model_3 = smf.ols(formula='mpg ~ weight + displacement + horsepower + acceleration', data=data)
result_3 = model_3.fit()
print(result_3.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     233.4
Date:                Sun, 01 Oct 2023   Prob (F-statistic):          9.63e-102
Time:                        20:14:50   Log-Likelihood:                -1120.6
No. Observations:                 392   AIC:                             2251.
Df Residuals:                     387   BIC:                             2271.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       45.2511      2.456     18.424   